In [ ]:
from gym import Env
from gym.spaces import Discrete, Box

import numpy as np
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

from py4j.java_gateway import JavaGateway


In [ ]:
# Conecta no servidor de treinamento

gateway = JavaGateway()
minitruco_java = gateway.entry_point


In [ ]:
# Env customizado do gym que encapsupla o servidor

class MinitrucoEnv(Env):
    def __init__(self):
        # Ações que podemos tomar: jogar a carta 0, a carta 1 ou a carta 2
        self.action_space = Discrete(3)

        # Espaço de observação: os 3 primeiros elementos são as cartas em mãos,
        # o restante são os valores das cartas na mesa, pontuações, etc.
        # Detalhes em SituacaoJogo.java#ranges e #toObservacao
        low, high = np.array(minitruco_java.ranges()).transpose()
        self.observation_space = Box(low=np.array(low), high=np.array(high), dtype=np.float32)

        self.episodio = None
        self.state = None

    def _get_obs(self):
        values = self.episodio.estado().split()
        return [float(val) for val in values]
        # return np.array(float_values[24:27], dtype=np.float32)

    def step(self, action):
        action = int(action)
        last_observation = self.state
        cartaJogada = last_observation[action]
        if cartaJogada == -1:
            # Jogada inválida (não deve acontecer no Keras por causa do masking, mas
            # se rolar por conta do teste inicial, só ignora e mantém no mesmo estado)
            return last_observation, 0, False, {}

        self.episodio.executa(action)
        estado_str = self.episodio.estado()
        if estado_str == "EQUIPE 1 VENCEU":
            return last_observation, 1.0, True, {}
        if estado_str == "EQUIPE 2 VENCEU":
            return last_observation, -1.0, True, {}

        self.state = self._get_obs()
        ganhoPontosEquipe1 = self.state[13] - last_observation[13]
        ganhoPontosEquipe2 = self.state[14] - last_observation[14]

        # Dá 1 ponto de recompensa se a equipe 1 ganhou pontos e -1 se a equipe 2 ganhou pontos
        reward = ganhoPontosEquipe1 - ganhoPontosEquipe2
        terminated = False

        return self.state, reward, terminated, {}

    def render(self):
        pass

    def reset(self, seed=None, options=None):
        if self.episodio is not None:
            self.episodio.finaliza()

#         super().reset(seed=seed) # required by check_env

        self.episodio = minitruco_java.novoEpisodio()
        self.state = self._get_obs()

        return self.state


env = MinitrucoEnv()


In [ ]:
# Roda alguns episódios com política "jogue uma carta aleatoriamente"

import time

episodes = 50
total_reward = 0
for episode in range(1, episodes+1):
    state = env.reset()
    terminated = False
    score = 0
    start = time.time()

    while not terminated:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, terminated, info = env.step(action)
        score+=reward
    total_reward += score
    print('Episode:{} Score:{} Time:{}s'.format(episode, score, time.time() - start))
print("Average reward per episode {}".format(total_reward/episodes))


In [ ]:
states = env.observation_space.shape
actions = env.action_space.n

def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states[0])))
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

model = build_model(states, actions)

model.summary()


In [ ]:
class MaskingDQNAgent(DQNAgent):
    def __init__(self, *args, **kwargs):
        super(MaskingDQNAgent, self).__init__(*args, **kwargs)

    def compute_q_values(self, state):
        q_values = super().compute_q_values(state)
        mask = np.array([1 if self._is_action_valid(state, action) else -np.inf for action in range(self.nb_actions)])
        masked_q_values = q_values + mask
        return masked_q_values

    def _is_action_valid(self, state, action):
        return state[0][action] != -1

def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = MaskingDQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=1000, target_model_update=1e-2)
    return dqn

dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
# Original era 50K steps, vamos devagar
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)


In [ ]:
scores = dqn.test(env, nb_episodes=50, visualize=False)
print("Average reward per episode: {}".format(np.mean(scores.history['episode_reward'])))
